In [1]:
import numpy as np
import pandas as pd
import os
import sys
CURRENT_DIR = os.path.dirname(os.getcwd())
sys.path.append(os.path.dirname(CURRENT_DIR+'/func'))
from func.photonic_func import Photonic

In [2]:
import numpy as np
from plotly.offline import init_notebook_mode, iplot 
import plotly.graph_objs as go
init_notebook_mode(connected=True)  # for Jupyter Lab notebook

photonic = Photonic(config='fake_tof_night_1550')
dist_vec = np.array([0.2,0.5,1,2,5,10])
trace0 = go.Scatter(x=dist_vec,
                    y=1000 * photonic.wallFlux(dist_vec=dist_vec), mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                    name='wallFlux [W/m**2]')
trace01 = go.Scatter(x=dist_vec,
                    y=1000 * photonic.wallFlux(dist_vec=dist_vec, light_type='solar'), mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                    name='Solar wallFlux [W/m**2]')
trace1 = go.Scatter(x=dist_vec,
                    y=1000 * photonic.siliconFlux2(dist_vec=dist_vec), mode='lines+markers',
                    name='siliconFlux [W/m**2]')
trace2 = go.Scatter(x=dist_vec,
                    y=photonic.photoelectron2(dist_vec=dist_vec), mode='lines+markers',
                    name='photoelectrons / burst')
trace3 = go.Scatter(x=[1],
                    y=[0.01], mode='text', textposition='top right',
                    name='text', text=['Photonic simulation of light created by a light source attached to a camera<br>'
                    +'1. Flux is calculated on a wall at a certain distance assuming CW lighting mode<br>'
                    +'2. Flux on the focal plane of the silicon sensor as imaged from the wall thru the lens<br>'
                    +'3. Photoelectrons per a single burst collect in the photodiode of the pixel'])

data = [trace0, trace01, trace1, trace2, trace3]

layout = dict(title='Photonic simulation - Flux on wall/sensor and PE count',
              xaxis=dict(title='Wall Distance [m]', type='log'),  # Select 'log' or 'linear'
              yaxis=dict(title='Flux [mW/m**2], PE [e-]',type='log'),  # Select 'log' or 'linear'
              template='plotly_dark',
              barmode='group',
              hovermode='x')

iplot(dict(data=data, layout=layout))

In [22]:
test_cfg = ['fake_tof_night_850','fake_tof_day_850',
            'fake_tof_night_940','fake_tof_day_940',
            'fake_tof_night_1375','fake_tof_day_1375',
            'fake_tof_night_1550','fake_tof_day_1550']
data = [0,1,2,3,4,5,6,7, 8, 9, 10, 11, 12, 13, 14, 15]
for i, cfg in enumerate(test_cfg):
    name1 = cfg+' IR'
    photonic = Photonic(config=cfg)
    dist_vec = np.array([0.2,0.5,1,2,5,10])
    trace0 = go.Scatter(x=dist_vec,
                        y=1000 * photonic.wallFlux(dist_vec=dist_vec),   # Select 'lines', 'markers' or 'lines+markers'
                        mode='lines+markers', name=name1)
    trace1 = go.Scatter(x=dist_vec,
                        y=1000 * photonic.wallFlux(dist_vec=dist_vec, light_type='solar'), mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                        name=(cfg+' Solar'))
    data[2*i] = trace0
    data[2*i+1] = trace1
#     print(i, data)



layout = dict(title='Photonic simulation - Flux on wall',
              xaxis=dict(title='Wall Distance [m]', type='log'),  # Select 'log' or 'linear'
              yaxis=dict(title='Flux [mW/m**2]',type='log'),  # Select 'log' or 'linear'
              template='plotly_dark',
              barmode='group',
              hovermode='x')

iplot(dict(data=data, layout=layout))

SyntaxError: invalid syntax (<ipython-input-20-37de783a0b22>, line 1)